# EDA v2

In [1]:
import os
import gc
import warnings

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
import statsmodels as sm
# import xgboost as xgb
from prettytable import PrettyTable

In [3]:
import plotly.figure_factory as ff
import plotly.express as px

In [4]:
from sklearn.preprocessing import (
    OrdinalEncoder
)
from sklearn.decomposition import (
    PCA
)
from sklearn.pipeline import (
    Pipeline,
    make_pipeline,
)


## (A) Import Data

In [5]:
PATH = os.getcwd()
print(PATH)
PATH = PATH.split('/')[:-2]
PATH = ''.join([str(folder + '/') for folder in PATH])
print(PATH)

/Users/school/Documents/repositories/Datasets_EDA/src/Allstate Claims Severity/code/v2
/Users/school/Documents/repositories/Datasets_EDA/src/Allstate Claims Severity/


### A.1

*'train.csv'* file does go on longer then line 188,320. But there appears to be some formatting issue after that line. I tried to write a separate python file that can add line by line of *'train.csv'* into a Panda's Dataframe but that seem to take to long. So I'll use up to 188,320 for now.

In [6]:
train_df = pd.read_csv(
    filepath_or_buffer= os.path.join(PATH, 'train.csv'),
    on_bad_lines= 'skip' # issue on line 188320
)

In [7]:
print(train_df.iloc[188317:188319]) # need to drop 188318 as it doesn't fit with data 

                   id      cat1      cat2      cat3      cat4      cat5  \
188317  587633.000000         B         A         A         B         A   
188318       0.348388   .672862   .551054   .344450   .447670   .538810   

            cat6      cat7      cat8      cat9  ...     cont6     cont7  \
188317         A         A         A         A  ...  0.844563  0.533048   
188318   .492200   .481306   .654753   .406237  ...       NaN       NaN   

          cont8    cont9   cont10    cont11    cont12    cont13   cont14  \
188317  0.97123  0.93383  0.83814  0.932195  0.946432  0.810511  0.72146   
188318      NaN      NaN      NaN       NaN       NaN       NaN      NaN   

           loss  
188317  4751.72  
188318      NaN  

[2 rows x 132 columns]


In [8]:
train_df.drop(
    inplace= True,
    index= 188318,
    axis= 'index'
)
print(train_df.iloc[188317:188319]) # need to drop 188318 as it doesn't fit with data

              id cat1 cat2 cat3 cat4 cat5 cat6 cat7 cat8 cat9  ...     cont6  \
188317  587633.0    B    A    A    B    A    A    A    A    A  ...  0.844563   

           cont7    cont8    cont9   cont10    cont11    cont12    cont13  \
188317  0.533048  0.97123  0.93383  0.83814  0.932195  0.946432  0.810511   

         cont14     loss  
188317  0.72146  4751.72  

[1 rows x 132 columns]


### A.2

Combine rest of file

In [9]:
with open(file= os.path.join(PATH, 'train.csv'), mode= 'r', newline= '\n') as f:
    file_reminder = list()
    
    for i, line in enumerate(f):
        if i < 188320 or i % 2 == 0:
            continue
        
        new_line: list[str] = line.split(',')[:-1]
        new_line = [s.replace(' ','') for s in new_line] 
        new_line = [s.replace(',','') for s in new_line]
        file_reminder.append(new_line)

    temp_df = pd.DataFrame(
        data= file_reminder,
        columns= train_df.columns
    )

In [10]:
print(temp_df)

          id cat1 cat2 cat3 cat4 cat5 cat6 cat7 cat8 cat9  ...    cont6  \
0     567844    B    A    A    A    A    B    A    A    A  ...  .904450   
1     567845    A    B    B    A    A    A    A    A    B  ...  .470691   
2     567847    A    B    A    A    A    A    A    A    B  ...  .741088   
3     567848    A    B    A    A    B    A    A    A    B  ...  .456654   
4     567854    B    A    A    B    A    B    A    A    A  ...  .499177   
...      ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...      ...   
6269  587620    A    B    A    A    A    A    A    A    B  ...  .242437   
6270  587624    A    A    A    A    A    B    A    A    A  ...  .334270   
6271  587630    A    B    A    A    A    A    A    B    B  ...  .345883   
6272  587632    A    B    A    A    A    A    A    A    B  ...  .704364   
6273  587633    B    A    A    B    A    A    A    A    A  ...  .844563   

        cont7    cont8    cont9   cont10   cont11   cont12   cont13   cont14  \
0     .633885  .829

In [11]:
train_df= pd.concat(
    objs=[train_df, temp_df]
)
del temp_df

In [12]:
print(train_df)

          id cat1 cat2 cat3 cat4 cat5 cat6 cat7 cat8 cat9  ...     cont6  \
0        1.0    A    B    A    B    A    A    A    A    B  ...  0.718367   
1        2.0    A    B    A    A    A    A    A    A    B  ...  0.438917   
2        5.0    A    B    A    A    B    A    A    A    B  ...  0.289648   
3       10.0    B    B    A    B    A    A    A    A    B  ...  0.440945   
4       11.0    A    B    A    B    A    A    A    A    B  ...  0.178193   
...      ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...       ...   
6269  587620    A    B    A    A    A    A    A    A    B  ...   .242437   
6270  587624    A    A    A    A    A    B    A    A    A  ...   .334270   
6271  587630    A    B    A    A    A    A    A    B    B  ...   .345883   
6272  587632    A    B    A    A    A    A    A    A    B  ...   .704364   
6273  587633    B    A    A    B    A    A    A    A    A  ...   .844563   

         cont7    cont8    cont9   cont10    cont11    cont12    cont13  \
0      0.335

In [13]:
print(train_df.shape)

(194592, 132)


### A.3

- Define useful constants
- Fix column types

In [14]:
CAT_X = [str('cat' + str(i)) for i in range(1, 117, 1)]
CONT_X = [str('cont' + str(i)) for i in range(1, 15, 1)]
Y = 'loss'

CAT_X_Y, CONT_X_Y = CAT_X.copy(), CONT_X.copy()
CAT_X_Y.append(Y)
CONT_X_Y.append(Y)

In [15]:
# https://pandas.pydata.org/pandas-docs/stable/user_guide/basics.html#basics-dtypes

train_df['id']= train_df['id'].astype('int32')
train_df['loss']= train_df['loss'].astype('float32')

train_df[CAT_X]= train_df[CAT_X].astype('string')
train_df[CONT_X]= train_df[CONT_X].astype('float32')

print(train_df.dtypes)

id                 int32
cat1      string[python]
cat2      string[python]
cat3      string[python]
cat4      string[python]
               ...      
cont11           float32
cont12           float32
cont13           float32
cont14           float32
loss             float32
Length: 132, dtype: object


In [16]:
gc.collect()

0

## (B) Data Cleaning

- Missing values
- Duplicate data
- Outliers
- Data Quality Measures
    - Accuracy
    - Completeness
    - Timeliness
    - Believability
    - Interoperability

### B.1 Missing Data

In [17]:
null_rows = train_df.isnull().any(axis=1)
print(null_rows.describe()) # seems to be no missing data

del null_rows

count     194592
unique         1
top        False
freq      194592
dtype: object


### B.2 Duplicate Rows

In [18]:
dup_rows = train_df.duplicated()
print(dup_rows.describe()) # no duplicates

del dup_rows

count     194592
unique         2
top        False
freq      188318
dtype: object


### B.3 Remove Outliers

In [19]:
# Did not do

### B.4 Data Quality Measures
- Accuracy
- Completeness
- Timeliness
- Believability
- Interoperability

Not knowing what the columns are makes it hard to preform this assessment.

In [20]:
gc.collect()

0

## (C) Graphs
- Correlation Map
- Boxplot
- Histogram
- Quantile Plot
- Scatter Plot (correlation)
- Mean, Median, Mode, etc...

In [21]:
# Not doing rn

In [22]:
gc.collect()

0

1. Identify
    - *Target*
    - *Predictors* & their *Exposure*

2. Visuals of *Predictors* & *Target*

3. Frequency tables for *Categorical/Nominal* *Predictors*

4. Start creating different models (in different notebooks)

## (D) Save new DF

In [23]:
train_df.to_csv(
    path_or_buf= os.path.join(PATH, 'code', 'v2', 'train_v2.csv'),
    index= False
)